In [1]:
%cd yolov5


c:\Users\pande\Downloads\Project_2023\yolov5


In [2]:
%pip install torch



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# %apt-get update && apt-get install libgl1 -y

In [6]:
%pip install torchvision


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import torchvision.transforms as transforms


In [10]:
import os
from PIL import Image
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Define the path to your dataset
dataset_root = "C:/Users/pande/Downloads/Project_2023/Dataset"  # Adjust this path as per your dataset structure

# Define the transformations for data augmentation
train_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])

# Custom dataset class to load images and adjust labels
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        # Load filenames and labels
        self.image_dir = os.path.join(root_dir, 'images', 'train')  # Adjust this according to your dataset structure
        self.label_dir = os.path.join(root_dir, 'labels', 'train')  # Adjust this according to your dataset structure
        self.filenames = os.listdir(self.image_dir)
        self.labels = {}  # Assuming labels are stored in a dictionary

        # Load labels from file or database
        self.load_labels()

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        # Load image
        img_name = os.path.join(self.image_dir, self.filenames[idx])
        image = Image.open(img_name).convert('RGB')

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Load corresponding label
        label = self.labels[self.filenames[idx]]

        # Adjust labels if data augmentation is applied
        if self.transform:
            label = self.adjust_labels(label, image.shape[1], image.shape[2])

        return image, label

    def load_labels(self):
        # Load labels from file or database
        pass  # Implement this function to load labels

    def adjust_labels(self, label, img_width, img_height):
        # Adjust bounding box coordinates based on image transformations
        class_name, x_min, y_min, x_max, y_max = label.split()
        x_min, y_min, x_max, y_max = float(x_min), float(y_min), float(x_max), float(y_max)

        # Example: Scale coordinates based on image size
        # Adjusting for resized image (assuming original image size is 224x224)
        x_min = x_min * img_width / 224
        y_min = y_min * img_height / 224
        x_max = x_max * img_width / 224
        y_max = y_max * img_height / 224

        return f"{class_name} {x_min} {y_min} {x_max} {y_max}"  # Adjusted label format

# Instantiate the dataset with augmentation
dataset = CustomDataset(root_dir=dataset_root, transform=train_transforms)

# Define data loader
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Iterate over the dataset
for images, labels in train_loader:
    # Perform training steps here
    pass  # Placeholder for training steps


KeyError: 'Normal3586.png'

In [30]:
!python train.py --img-size 224 --batch-size 60 --epochs 50 --data dataset.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt


train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=60, imgsz=224, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
fatal: cannot change to '/workspace/Nishu': No such file or directory
YOLOv5 🚀 2023-12-18 Python-3.8.10 torch-2.1.2+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40536MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, 

In [31]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
Image(filename='runs/train/exp2/test_batch2_pred.jpg', width=1000)

In [ ]:
#!python detect.py --source runs/train/exp/testimg.jpg --weights runs/train/exp/weights/best.pt --conf 0.25

!python detect.py --source runs/train/exp2/a.jpg --weights best.pt

In [ ]:
Image(filename='runs/detect/exp4/a.jpg', width=416)

In [ ]:
from utils.plots import plot_results 
plot_results(save_dir='runs/train/exp2')  # plot results.txt as results.png